# Web Scraping Reports from " www.sustainability-reports.com "

Website: https://www.sustainability-reports.com/

In [104]:
# Importing required libraries
import requests
import pandas as pd
import os
import urllib.request
from bs4 import BeautifulSoup

#### Creating dataframe with report details

In [130]:
df=pd.DataFrame(columns=['company_name' , 'country', 'company_sector', 'report_link', 'report_name', 'report_year', 'language'])
URL= "https://www.sustainability-reports.com/annual-reports/"
html_text = requests.get(URL).text
soup = BeautifulSoup(html_text, 'html.parser')
company = soup.find_all("div", class_="col col1")
#print(company)
i=0
for a in company:
  x=a.find("a", href=True)
  company_data_link = x['href']
  company_name = x.text
  #print(company_name)
  company_data=requests.get(company_data_link).text
  company_data_soup = BeautifulSoup(company_data, 'html.parser')
  report_data=company_data_soup.find_all("div", class_="col col1")
  company_sector = report_data[0].text.split('\t')[2]
  cntry = report_data[0].text.split('\t')[3]
  country = cntry.split(':')[1]
  #print(country)
  for a in report_data[1:]:
    x = a.find('a', href=True)
    y = a.find('img')
    report_year=a.find('h2').text
    report_language = y.attrs['src'].split('/')[-1]

    # extract pdf reports in english  
    if report_language=="gb.png" and  x['href'].split('.')[-1]=="pdf":
      language = 'en'
      report_link = x['href']
      report_name = x.text
      to_append = [company_name , country, company_sector, report_link, report_name, report_year, language]
      print(i,'  :  ',to_append)
      df.loc[len(df)] = to_append
      i=i+1
    
  """
  # Checking for 1 loop
  if i>1:
    break"""



0   :   ['ABN AMRO', ' The Netherlands ', 'Financials ', 'https://downloads.ctfassets.net/1u811bvgvthc/62067Em47HIFTg3hGxdQHY/3fcd6a3ac35ba44be18fad01cc84b7c0/ABN_AMRO_____Integrated_Annual_Report_2021.pdf', 'Integrated Annual Report 2021', '2021', 'en']
1   :   ['ABN AMRO', ' The Netherlands ', 'Financials ', 'https://assets.ctfassets.net/1u811bvgvthc/ZPL95gLg9RUZs6WODxSZq/054ab555339ac72bead687d2c9dc066b/ABN_AMRO_____Integrated_Report_2020.pdf', 'Integrated Report 2020', '2020', 'en']
2   :   ['ABN AMRO', ' The Netherlands ', 'Financials ', 'https://www.abnamro.com/nl/images/Documents/010_Over_ABN_AMRO/Annual_Report/2019/ABN_AMRO_Integrated_Annual_Review_2019.pdf', 'Integrated Annual Review 2019', '2019', 'en']
3   :   ['ABN AMRO', ' The Netherlands ', 'Financials ', 'https://www.abnamro.com/en/images/Documents/010_About_ABN_AMRO/Annual_Report/2018/ABN_AMRO_Integrated_Annual_Review_2018.pdf', 'Integrated Annual Review 2018', '2018', 'en']
4   :   ['ABN AMRO', ' The Netherlands ', 'Fi

In [131]:
df.head()

,company_name,country,company_sector,report_link,report_name,report_year,language
0,ABN AMRO,The Netherlands,Financials,https://downloads.ctfassets.net/1u811bvgvthc/6...,Integrated Annual Report 2021,2021,en
1,ABN AMRO,The Netherlands,Financials,https://assets.ctfassets.net/1u811bvgvthc/ZPL9...,Integrated Report 2020,2020,en
2,ABN AMRO,The Netherlands,Financials,https://www.abnamro.com/nl/images/Documents/01...,Integrated Annual Review 2019,2019,en
3,ABN AMRO,The Netherlands,Financials,https://www.abnamro.com/en/images/Documents/01...,Integrated Annual Review 2018,2018,en
4,ABN AMRO,The Netherlands,Financials,https://www.abnamro.com/nl/images/Documents/05...,Annual Report 2017,2017,en


In [132]:
len(df)

115

In [133]:
df.tail()

,company_name,country,company_sector,report_link,report_name,report_year,language
110,Van Lanschot Kempen,The Netherlands,Financials,https://media.vanlanschot.nl/media/pdfs/annual...,Annual Report 2020,2020,en
111,Van Lanschot Kempen,The Netherlands,Financials,https://media.vanlanschot.nl/media/pdfs/annual...,Annual Report 2019,2019,en
112,Van Lanschot Kempen,The Netherlands,Financials,https://media.vanlanschot.nl/media/pdfs/2018-a...,Annual Report 2018,2018,en
113,Van Lanschot Kempen,The Netherlands,Financials,https://media.vanlanschot.nl/media/pdfs/van-la...,2016 Annual Report,2016,en
114,VP Capital,The Netherlands,Financials,https://vpcapital.eu/wp-content/uploads/2021/0...,Progress Report 2020,2020,en


In [134]:
df1=df.copy()

#### Extracting dataframe with "no error links"

In [135]:
# Removing error links from dataframe
j=0
for report in df1['report_link'].tolist():
  try:
    r = requests.get(report, timeout=10)
    print('Link',j,'checked', ": No Error")
    
  except requests.exceptions.RequestException as e:
    df1 = df1[df1.report_link != report].reset_index(drop=True) 
    print ("Link",j,'checked',': Error :',SystemExit(e), ": Removed link from dataframe")
  
  j=j+1

Link 0 checked : No Error
Link 1 checked : No Error
Link 2 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 3 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 4 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 5 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 6 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 7 checked : Error : HTTPSConnectionPool(host='www.abnamro.com', port=443): Read timed out. (read timeout=10) : Removed link from dataframe
Link 8 checked : No Error
Link 9 checked : No Error
Link 10 checked : No Error
Link 

In [142]:
print("Length of dataframe before removing links with error :",len(df))
print("Length of dataframe after removing links with error :",len(df1))

Length of dataframe before removing links with error : 115
Length of dataframe after removing links with error : 106


In [143]:
df1.head()

,company_name,country,company_sector,report_link,report_name,report_year,language
0,ABN AMRO,The Netherlands,Financials,https://downloads.ctfassets.net/1u811bvgvthc/6...,Integrated Annual Report 2021,2021,en
1,ABN AMRO,The Netherlands,Financials,https://assets.ctfassets.net/1u811bvgvthc/ZPL9...,Integrated Report 2020,2020,en
2,AgroFair,The Netherlands,Agri,https://www.agrofair.nl/wp-content/uploads/Agr...,Sustainability Report 2020,2020,en
3,AgroFair,The Netherlands,Agri,https://www.agrofair.nl/wp-content/uploads/Agr...,Sustainability Report 2019,2019,en
4,AgroFair,The Netherlands,Agri,https://www.agrofair.nl/wp-content/uploads/Agr...,Sustainability Report 2018,2018,en


#### Downloading reports 

In [165]:
# Downloading reports and adding downloaded file name to dataframe
path='/content/sustainability_reports'
os.makedirs(path,exist_ok=True)
k=1
for report in df1['report_link'].tolist():
  try:
    urllib.request.urlretrieve(report,path+'/'+os.path.basename(report))
    
    ind=df1.loc[df1['report_link']==report].index
    df1.loc[ind,'downloaded_file']=os.path.basename(report)
    print(df1.loc[ind,'downloaded_file'])
    print(f"{os.path.basename(report)} file downloaded to {path}")
  except:
        pass


0    ABN_AMRO_____Integrated_Annual_Report_2021.pdf
Name: downloaded_file, dtype: object
ABN_AMRO_____Integrated_Annual_Report_2021.pdfDownloaded files to /content/sustainability_reports
1    ABN_AMRO_____Integrated_Report_2020.pdf
Name: downloaded_file, dtype: object
ABN_AMRO_____Integrated_Report_2020.pdfDownloaded files to /content/sustainability_reports
2    AgroFair_Sustainability_report_2020_Lres.pdf
Name: downloaded_file, dtype: object
AgroFair_Sustainability_report_2020_Lres.pdfDownloaded files to /content/sustainability_reports
3    AgroFair_Sustainability_Raport_2019_lr.pdf
Name: downloaded_file, dtype: object
AgroFair_Sustainability_Raport_2019_lr.pdfDownloaded files to /content/sustainability_reports
4    AgroFair_Sustainability_Report_2018_lr.pdf
Name: downloaded_file, dtype: object
AgroFair_Sustainability_Report_2018_lr.pdfDownloaded files to /content/sustainability_reports
5    AgroFair_sustainability_report_2018_online.pdf
Name: downloaded_file, dtype: object
AgroFair_s

In [166]:
df1['downloaded_file']

0      ABN_AMRO_____Integrated_Annual_Report_2021.pdf
1             ABN_AMRO_____Integrated_Report_2020.pdf
2        AgroFair_Sustainability_report_2020_Lres.pdf
3          AgroFair_Sustainability_Raport_2019_lr.pdf
4          AgroFair_Sustainability_Report_2018_lr.pdf
                            ...                      
101        annual-report-2020-van-lanschot-kempen.pdf
102        annual-report-2019-van-lanschot-kempen.pdf
103        2018-annual-report-van-lanschot-kempen.pdf
104               van-lanschot-2016-annual-report.pdf
105             VPCAPITAL-ProgressReport2020-EN_I.pdf
Name: downloaded_file, Length: 106, dtype: object

In [167]:
df1['downloaded_file'].isna().sum()

37

In [168]:
df1=df1.dropna()

In [169]:
len(df1)

69

In [170]:
df1['downloaded_file'].isna().sum()

0

#### Creating list of reports with details

In [172]:
df1.to_csv('list_of_reports.csv')